In [31]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [49]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
#op_model = "AV_IN_Model"
op_model = "PowerLaw"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

#strat_model = "Constant"
strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

In [50]:
#### parameters of the agents ####
p = [2.0, 1.7, 2.0, 1.0, 0.3, 0.0, 0.0, 0.1, 5.0, 1.0, 0.0, 0.0, 0.8, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
N = 8
system_size = (20.0, 20.0) #important for periodic boundaries

### of the simulation ###

# simulation time-step #
δt = 0.01
sim_p = [0.0, 10.0, 0.05, δt, p[1]*p[2]] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"


### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = create_geometry_closed_room(10.0, 10.0, 0.3)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

R_ = 5.0
p0 = (10.0, 10.0)
Init_Antipodal!(menge, R_, p0)

In [51]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [52]:
using GR
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(4,16, 4, 16)
        GR.setviewport(0.0, 1, 0.0, 1)       

        plot_agents(positions[j, :], headings[j, :], 2.1, 100, 0.7, 999, 0.1)

        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.7, 0.95, timer)
        GR.text(0.7, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [53]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAGZHltZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAQVmWIhAAX//73x0/Apuj5ZsudRT0hMIrY+kgroIA+awfyGwAAAwAAAwAAAwAM8Sa9KqL6uLjLQAAaYaZyrB4dXACpiV/zJKkLR9d0cJJoxKsYjwHhcF0TVoI4b0omfRXhP2L5UWZlY2cls2QzQsiImDU08KOiOhrEnbH9Y7IvfqOVGQhM8yM8baSSg6UxJ7jGtoP8kmb9o0kMNBjvVwtyBQmRK92wDXYRsbi+Dvl9Ro9hqPuDjuakSLotMrq2KiJf3Gd3fyjm07z2FA5/GSDA23UUAOzVowc3PBM35jf9d0ixz/ipjtK5xArgafYbkewDbnRGlPXU/O8xOsxOltNtrG2QiEb8bIuMeZkXrOmNq//dCydBKhJ9NTSiCXqpJjgk25ldpaR3MkbtyozD27gMdiGadGcbuN+SyZ+lTyTA5hwdZtky7fnNZ76/3L7PNPttnjlgGGe3dnTCCPTWrW/5l/m3bn82AGuQP8shJnG0fsLU0Dr6jn7G/2Iz7/nBHB4zhlDGaVxKZDyeUSV+KWPCPV1z8pXU4BrgJaZgoVxbRttr5J9G6QhiRNuqeP////9jvV94QqqMB7ZNVWsMawEvcaiOffUHPjqWzeO3WTUCUU6TsWAeFwxnAPsNGUnx/rGo6L5xOXnKhdLGOGF8+rSNqvGE0raH91vOEIAcExeL63QbvfZXoWnj6uJNWXJTTO6CN8WpnXBDpyDgcmwp43BvO9c5HTWyz6nvP1b5PxPKDl3yNBP/I2x3CPR19xmLELUANqro6nBV706LIkOrXy4IX969XxOpkBh+9kX+7yrz5Psg3PlyuCqOpfpIexNxXiFYOtcu45/doVXJigzwevp5Lc+kl5qkf1NDOkC7ZQ/PRk3EOdOVSV6gGtUmNOrlvf8oKdC2jq/JPffMTaQ+6Ci4PbI+e80Yl+Xvs7fl03M50ZySp313ACu1nyP+b1HYZpKgE7k5dsbeLrLVjmKyk2Bb1eYL/+oNJLAKKSirJ778j0SwtIOsoDfvllmOT63z0Rr0t0+KWb0MMyPyxq0pKRFdf0ndeZ8un02bQCHUU1x0XMT2wVnL9O7RRuECPQbABvbLJXeaMoKxfJj6+aq4bCAt53MQ6C5KNugU0ID5jf33PnmvNQDwCQV/EPsnYuOEU1//z3zj70TGwiwq1CtRsBqyx9oYiA88PzH/1yabRr/tcXExvy80qwrc2QNjK9LzKRnfF7CTwFd6rhHRZ6Ty4SdfdqBXkebx0lyeC9QttBULU7SoSTWftAm1KG8L20YGUonYXtsDQWfG/f4oyhOPCln3EJUapejVdQ1BxqfdjQZ3+SG5K1E45mhcpE1ihww0dsxE/ewItqb//PNSaAtwMF6W4jTbBexGH8qQ9FXF6xpLGqoMFmjBm60yapsO2t+yWtZBq9WhRAjKFtIWxpV5CJFwqmXuJjf2V8zYJWxFdwOl6Jac8wqVucbca09zgf9oqGWjAZY1B8vaEzqMXJNs4stC+ToE+nDJbqaWLs6ReJsLjtLEtJd/+Sy8e1Z3zYVY9pn6tRO7O8l/k1Ppp2IGxcTQk9L8QxwsSjvtsvglptN7D75i5EKuiGb7kO8kPG07w9PSrUCqgvs3uKOS83f6zOtDsfV+rDiZxeZHndzZxBovYoYHJ0Niv5fYNSHORRVpKN6QzGsDf1hCx8hvh7qISsGE2GtOzRTMNinDeUeD4sSLgGeL2YpiKgbSDPeTnbjQLw4Wk4XrkEpm0O+tvC9P1nnCbEh1Sl0C/wLQbNo0W7wxX5H3DFwsOkkwgf+tqar8rw0TZ7GJIGziSP1qNH79vNgz9hUpvpj1PRVzlnOfXKcy7Nw34qgAkpmMx++eQI6MdvptGIpAXEWyi0vcbr4rQmvnLSQU09W3TquIh3XYouIMXujoXtMpFJR1VuuHMwZvU7Tb1JWhtvjmjEmmym+Av52ANMCLJQOhcfiD2td5jLLeqiIB6aHKR1PjUul0A1ZGgb6r7+s614EMGex/O6hB3nhCBrV/BbiIiNOaqbgPdWuJofEN2FOBCCrIIHn5u4B5P1XRaJdKyTsHo+wnDsjiL5A0B0RXkxebpJHtaBwYR0L3fMkBucVMYFM2JRfcBe5EbDNgj7b1KU140T4oldFHDwaYE/L+yzGtefhKl+w/9ZXwA4OtSyC+Id6aerXE4JlqJwwbdscjj1mK2B8FXUtQRrzKvj4E0lZT1zrf/rVHL2Jo/40dFkYLQheEgDQDyMFneDR3utVrJHCxKN4li3lbuvpAM+0xICwd73AVgY/YwXs9FDAOZgxBsVeDdvtOo2TvCDEL8p0ta996zzAam5/rCd7NRSWSwu10Pybqpsjo6sLr0yBIWmdXRIcgFudXseSEQfD/Ay8pgWfw1NWTKsYYa9sSWW441YK+6avTBp2g50FrnRzubgBzb841NXv/MYbBJIDy338hmMbK8Pfm6Jtt2pCGVC5qZdpmVJ1L3iRG3nHd35rxPlXU4IPzi/ydJThGUi6nqI/ZD/FwhKYKFpbcg8fLZN0NTMi3kDrvxHhlQ3eruXqbQ+2A8LUd7NQEqjoM18bxJJHAdIQJYM4YD1D/ItZj/359H7smns4Em6X+gRNJL3OPes02CjuIWf235IxKZQIPpNG33n9Pp1kOqeVe0tiFgK4sz0UPz5aPL3HzVt2l/VwJJDiNvcSIa8doTTM6X3pEZkBgQe0V1YDtVkajGph1+UvOnk8zTQt1hZ2Z6O6Z/rfVA5JXk29kr/b+cdht8Xl0/97NHq6Y8+VUNiupIyNbC8PNpMA4A/ZhxGlsDKWoW9YqI2zryjTzWkwk1ISg8cdy7EiCuGVecSvlnOdAYXL8Me2ctXmgKE1wni45TXM/74D4Gp8HnzL26BKYIJui3eZ/lUK5D1UM3mwTc3tEHq7ASI84liNoGhb6XTI3v+kLl2dwRoqUDMC8Jmc

In [84]:
### To save the results in a .txt file in the format ID/Frame/X/Y ###

Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Circle_8_AV_IN"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
### Perform Multiple Runs and Save the Output ###

samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [ ]:
### Save Initial Conditions for UMANS Software ###

name = "Circle_8"
path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/agents/"


#Set_Goal_from_E_des!(menge, (99.0, 0.0))
#Change_Coordinate_System_Agents!(menge, -1 .*(system_size./2))
Write_Agents_XML(menge, name, path)

In [ ]:
### Save Scenario for UMANS Software ###

path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/"
policy = "SocialForces"
agents = "Circle_8"

name = string(agents, policy)

Write_Scenario_XML(path, name, sim_p, system_size, policy, agents)